In [1]:
import tensorflow as tf
import keras as k
name = '/github/fer/model/models/ak31_32.h5'

# Ignore dropout at inference
k.backend.set_learning_phase(0)
model = k.models.load_model(name)

export_path = './convert_model/1'
#k.backend.get_session()
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors
# And stored with the default serving key
with k.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        export_path,
        inputs={'input_image': model.input},
        outputs={t.name:t for t in model.outputs})

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


AssertionError: Export directory already exists. Please specify a different export directory: ./convert_model/1

In [2]:
## show model signature

import os
#os.chdir('/python/')
os.chdir('./convert_model/1/')
! saved_model_cli show --dir ./ --all



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_image'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 48, 48, 1)
        name: input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['activation_4/Softmax:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: activation_4/Softmax:0
  Method name is: tensorflow/serving/predict


c:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import os
path_inception = '/Download\ksb_toolbox_v0_8.tar\ksb-csle\examples\models\inception\model/1/'
os.chdir(path_inception)
os.getcwd()
! saved_model_cli show --dir ./ --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict_images']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: ParseExample/ParseExample:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 5)
        name: index_to_string_Lookup:0
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: TopKV2:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: unknown_rank
        name: tf_example:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        sh

c:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Sanity check / prediction test using sample data 

! saved_model_cli run --dir ./ --tag_set serve --signature_def serving_default --inputs input_image=/python/data/x1.npy

c:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "c:\programdata\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\Scripts\saved_model_cli.exe\__main__.py", line 9, in <module>
  File "c:\programdata\anaconda3\lib\site-packages\tensorflow\python\tools\saved_model_cli.py", line 801, in main
    args.func(args)
  File "c:\programdata\anaconda3\lib\site-packages\tensorflow\python\tools\saved_model_cli.py", line 634, in run
    args.overwrite, tf_debug=args.tf_debug)
  File "c:\programdata\anaconda3\lib\site-packages\tensorflow\pyth

In [34]:
pwd()

'F:\\python\\convert_model\\1'

In [ ]:
# show model graph using tensor board / 경우에 따라 안되는듯? 1.민영/진영님 변환한 pb 는 잘 동작. 

from tensorflow.python.platform import gfile

with k.backend.get_session() as sess:
    #model_filename ='./output_graph.pb'
    model_filename = './convert_model/1/saved_model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='log'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

from tensorflow.python.saved_model import signature_def_utils
signature_def_utils.is_valid_signature()
#sess.graph_def # json 형태로 구조 확인